In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')

import joypy
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import colors, cm


In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [3]:
mouse = 'SNL_photo15_rewards'
date = '20200316'
saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
demod_trace_filename = mouse + '_' + date + '_' + 'smoothed_signal.npy'
restructured_data_filename = mouse + '_' + date + '_' + 'restructured_data.pkl'
demod_signal = np.load(saving_folder + demod_trace_filename)
trial_data = pd.read_pickle(saving_folder + restructured_data_filename) 

In [4]:
def get_photometry_around_event(all_trial_event_times, demodulated_trace, pre_window=2, post_window=2, sample_rate=10000):
    num_events = len(all_trial_event_times)
    event_photo_traces = np.zeros((num_events, sample_rate*(pre_window + post_window)))
    for event_num, event_time in enumerate(all_trial_event_times):
        try:
            plot_start = int(event_time*sample_rate) - pre_window*sample_rate
            plot_end = int(event_time*sample_rate) + post_window*sample_rate
            if plot_end - plot_start != sample_rate*(pre_window + post_window):
                print(event_time)
                plot_start = plot_start + 1
                print(plot_end - plot_start)
            event_photo_traces[event_num, :] = demodulated_trace[plot_start:plot_end]
        except: print(pre_window, event_time)
    return event_photo_traces

In [10]:
def get_mean_and_sem(trial_data, demod_signal):     
    events_of_int = trial_data.loc[(trial_data['State type'] == 4)]
        
    event_times = events_of_int['Time start'].values
    state_name = events_of_int['State name'].values[0]
    event_photo_traces = get_photometry_around_event(event_times, demod_signal)        
    event_photo_traces = get_photometry_around_event(event_times, demod_signal,  pre_window=5, post_window=5)
    norm_traces = stats.zscore(event_photo_traces.T, axis=0)


    sorted_traces = norm_traces.T

    x_vals = np.linspace(-2, 2, norm_traces.shape[0], endpoint=True, retstep=False, dtype=None, axis=0)
    y_vals = np.mean(sorted_traces, axis=0)
    sem = np.std(sorted_traces, axis=0)
    
    return x_vals, y_vals, sem, sorted_traces


In [6]:
def heat_map_and_mean(trial_data, demod_signal):
    x_vals, y_vals, sem, sorted_traces = get_mean_and_sem(trial_data, demod_signal)
    
    num_state_types = trial_data['State type'].unique().shape[0]
    fig, axs = plt.subplots(1, ncols=2, figsize=(10, 8))
    fig.subplots_adjust(hspace=0.5, wspace=0.2)
    
    fig.text(0.06, 0.02, mouse + ' ' + date, fontsize=12)


    axs[0].plot(x_vals, y_vals,lw=3,color='#3F888F')
    axs[0].fill_between(x_vals, y_vals-sem, y_vals+sem, alpha=0.5, facecolor='#7FB5B5', linewidth=0)
    
    axs[0].axvline(0, color='k', linewidth=2)
    axs[0].set_xlim([-2, 2])
    axs[0].set_xlabel('Time (s)')
    axs[0].set_ylabel('z-score')

    heat_im = axs[1].imshow(sorted_traces, aspect='auto', extent=[-3, 3, sorted_traces.shape[0], 0], cmap='rainbow')
    
    axs[1].axvline(0, color='w', linewidth=2)
    axs[1].tick_params(labelsize=10)
    axs[1].set_xlim([-2,2])
    axs[1].set_ylim([sorted_traces.shape[0], 0])
    axs[1].set_xlabel('Time (s)')
    axs[1].set_ylabel('Trial number (sorted)')
    vmin = sorted_traces.min()
    vmax = sorted_traces.max()
    edge = max(abs(vmin), abs(vmax))
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    heat_im.set_norm(norm)
    fig.colorbar(heat_im, ax=axs[1], orientation='horizontal', fraction=.1, label='z-score')
    
    axs[0].set_ylim(-2,4)
    return sorted_traces

In [13]:
events = heat_map_and_mean(trial_data, demod_signal)